In [2]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("GPU name:", torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "No GPU found")

CUDA available: True
GPU count: 1
Current device: 0
GPU name: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

Using: cuda


In [10]:
import os

print("easy ham", len(os.listdir("spamsassasin\\easy_ham")))
print("hard ham", len(os.listdir("spamsassasin\\hard_ham")))
print("spam", len(os.listdir("spamsassasin\\spam")))

easy ham 2551
hard ham 250
spam 501


Enron Dataset Preprocessing

In [ ]:
import pandas as pd

# Load your file
df = pd.read_csv("enron\\emails.csv") 
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

# Display first 3 rows
df.head(3)


Shape: (517401, 2)
Columns: ['file', 'message']


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...


In [13]:
for col in df.columns:
    print(f"\nColumn: {col}")
    print(df[col].iloc[0])



Column: file
allen-p/_sent_mail/1.

Column: message
Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [3]:
import re
import pandas as pd

def clean_enron_email(text):
    if pd.isna(text):
        return ""
    
    # 1. Split headers from body (body is after first blank line)
    parts = text.split("\n\n", 1)
    body = parts[1] if len(parts) > 1 else text

    # 2. Remove common header patterns accidentally left behind
    header_patterns = [
        r"Message-ID:.*\n",
        r"Date:.*\n",
        r"From:.*\n",
        r"To:.*\n",
        r"Subject:.*\n",
        r"Mime-Version:.*\n",
        r"Content-Type:.*\n",
        r"Content-Transfer-Encoding:.*\n",
        r"X-[\w-]+:.*\n",
    ]

    for pattern in header_patterns:
        body = re.sub(pattern, "", body)

    # 3. Remove excessive whitespace
    body = re.sub(r"\n+", "\n", body).strip()

    return body

# APPLY CLEANING
df['clean_text'] = df['message'].apply(clean_enron_email)

df[['clean_text']].head()


,clean_text
0,Here is our forecast
1,Traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"Randy,\n Can you send me a schedule of the sal..."
4,Let's shoot for Tuesday at 11:45.


In [4]:
print(df['clean_text'].iloc[0])

Here is our forecast


In [5]:
df = df[df['clean_text'].str.len() > 20]   # remove emails shorter than 20 chars
df = df.dropna(subset=['clean_text'])      # remove nulls

print("Remaining emails:", len(df))


Remaining emails: 510504


In [8]:
import os

os.makedirs("DATA/enron", exist_ok=True)

df[['clean_text']].to_csv("DATA/enron/enron_clean.csv", index=False)


In [9]:
print("Remaining emails:", len(df))
print(df.head())


Remaining emails: 510504
                       file  \
1    allen-p/_sent_mail/10.   
2   allen-p/_sent_mail/100.   
3  allen-p/_sent_mail/1000.   
4  allen-p/_sent_mail/1001.   
5  allen-p/_sent_mail/1002.   

                                             message  \
1  Message-ID: <15464986.1075855378456.JavaMail.e...   
2  Message-ID: <24216240.1075855687451.JavaMail.e...   
3  Message-ID: <13505866.1075863688222.JavaMail.e...   
4  Message-ID: <30922949.1075863688243.JavaMail.e...   
5  Message-ID: <30965995.1075863688265.JavaMail.e...   

                                          clean_text  
1  Traveling to have a business meeting takes the...  
2                     test successful.  way to go!!!  
3  Randy,\n Can you send me a schedule of the sal...  
4                  Let's shoot for Tuesday at 11:45.  
5  Greg,\n How about either next Tuesday or Thurs...  
